In [ ]:
import json
import matplotlib
from matplotlib import pyplot as plt
import pickle
import sklearn
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
%matplotlib inline

# Binned Data

In [ ]:
oresults_path = '../predicted-results/original/nbins-10'
mresults_path = '../predicted-results/mixed5050/nbins-10'
eresults_path = '../predicted-results/everything/nbins-10'

omodels_path = '../models/original'
mmodels_path = '../models/mixed5050/nbins-10'
emodels_path = '../models/everything/nbins-10'

test_sets = ['5050', '2575', 'everything']

mpoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 'all']
epoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 4000, 6000, 'all']



In [ ]:
with open(f'{emodels_path}/set_1/COF_all.pickle', 'rb') as cof:
    mmodel_cof = pickle.load(cof)

with open(f'{emodels_path}/set_1/intercept_all.pickle', 'rb') as f0:
    mmodel_f0 = pickle.load(f0)

In [ ]:
results = {'5050': {'COF': dict(),
                    'intercept': dict()},
           '2575': {'COF': dict(),
                    'intercept': dict()},
           'everything': {'COF': dict(),
                          'intercept': dict()}}
# First deal with the original models 
for tset in results:
    for target in ['COF', 'intercept']:
        with open(f'{omodels_path}/{target}.pickle', 'rb') as f:
            model = pickle.load(f)
        with open(f'{omodels_path}/{target}.ptxt', 'rb') as f:
            features = pickle.load(f)
        with open(f'{oresults_path}/{target}_on_{tset}.json', 'r') as f:
            data = json.load(f)
        results[tset][target]['original'] = {
#             'model': model,
#             'features': features,
#             'data': data,
              'n_train': len(model.oob_prediction_),
              'r_square': data[target]['r_square']
        }

# Then deal with the 5050 mixed models 
for tset in results:
    for i in range(5):
        for point in mpoints:
            for target in ['COF', 'intercept']:
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{mresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                if i == 0:
                    results[tset][target][f'5050_{point}'] = {
    #                 'model': model,
    #                 'features': features,
    #                 'data': data,
                      'n_train': len(model.oob_prediction_),
                      'r_square': [data[target]['r_square']]}
                else:
                    results[tset][target][f'5050_{point}']['r_square'].append(data[target]['r_square'])
# Lastly deal with the everything models
for tset in results:
    for i in range(5):
        for point in epoints:
            for target in ['COF', 'intercept']:
                with open(f'{emodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{emodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{eresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                if i == 0:
                    results[tset][target][f'everything_{point}'] = {
    #                 'model': model,
    #                 'features': features,
    #                 'data': data,
                      'n_train': len(model.oob_prediction_),
                      'r_square': [data[target]['r_square']]}
                else:
                    results[tset][target][f'everything_{point}']['r_square'].append(data[target]['r_square'])


In [ ]:
for tset in results:
    for target in ['COF', 'intercept']:
        for point in mpoints:
            # doi: dict of interest
            doi = results[tset][target][f'5050_{point}']
            doi['r_square_avg'] = np.average(doi['r_square'])
            doi['r_square_std'] = np.std(doi['r_square'])

        for point in epoints:
            doi = results[tset][target][f'everything_{point}']
            doi['r_square_avg'] = np.average(doi['r_square'])
            doi['r_square_std'] = np.std(doi['r_square'])
            

## Refined R<sup>2</sup> plot
### Mixed 5050 models on 5050 test set

In [ ]:
from numpy import *
from matplotlib.pyplot import *

In [ ]:
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.it'] = 'DejaVu Sans:italic'
rcParams['mathtext.bf'] = 'DejaVu Sans:italic:bold'


In [ ]:
markersize=6
COF_color = 'dodgerblue'
F0_color = 'firebrick'

In [ ]:
to_plot_on_5050 = {'COF': {'n_train': list(),
                           'r_square_avg': list(),
                           'r_square_std': list(),
                           'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['5050'][target].items():
        if '5050' in model:
            to_plot_on_5050[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_5050[target]['r_square_std'].append(data['r_square_std'])
            to_plot_on_5050[target]['n_train'].append(data['n_train'])
        elif 'original' in model: 
            to_plot_on_5050[target]['base'] = data['r_square']
                
plt.figure(figsize=(9, 6))

plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0, 3000)
plt.ylim(0.4, 1)

plt.errorbar(x=to_plot_on_5050['COF']['n_train'],
            y=to_plot_on_5050['COF']['r_square_avg'],
            yerr=to_plot_on_5050['COF']['r_square_std'],
            color=COF_color,
            fmt='o',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_5050['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_5050['intercept']['n_train'],
            y=to_plot_on_5050['intercept']['r_square_avg'],
            yerr=to_plot_on_5050['intercept']['r_square_std'],
            color=F0_color,
            fmt='s',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_5050['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)

legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/5050_on_5050.pdf', dpi=500)


# Refined R<sup>2</sup> plot 
### Everything models on 5050 test set

In [ ]:
to_plot_on_5050 = {'COF': {'n_train': list(),
                           'r_square_avg': list(),
                           'r_square_std': list(),
                           'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['5050'][target].items():
        if 'everything' in model:
            to_plot_on_5050[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_5050[target]['r_square_std'].append(data['r_square_std'])
            to_plot_on_5050[target]['n_train'].append(data['n_train'])
        elif 'original' in model: 
            to_plot_on_5050[target]['base'] = data['r_square']
                
plt.figure(figsize=(9, 6))

plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0, 8500)
plt.ylim(0.4, 1)
plt.errorbar(x=to_plot_on_5050['COF']['n_train'],
            y=to_plot_on_5050['COF']['r_square_avg'],
            yerr=to_plot_on_5050['COF']['r_square_std'],
            color=COF_color,
            fmt='o',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_5050['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_5050['intercept']['n_train'],
            y=to_plot_on_5050['intercept']['r_square_avg'],
            yerr=to_plot_on_5050['intercept']['r_square_std'],
            color=F0_color,
            fmt='s',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_5050['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)

legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/everything_on_5050.pdf', dpi=500)


# Refined R<sup>2</sup> plot 
### Mixed 5050 models on 2575 test set

In [ ]:
to_plot_on_2575 = {'COF': {'n_train': list(),
                           'r_square_avg': list(),
                           'r_square_std': list(),
                           'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['2575'][target].items():
        if '5050' in model:
            to_plot_on_2575[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_2575[target]['r_square_std'].append(data['r_square_std'])
            to_plot_on_2575[target]['n_train'].append(data['n_train'])
        elif 'original' in model: 
            to_plot_on_2575[target]['base'] = data['r_square']
                
plt.figure(figsize=(9, 6))

plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0, 3000)
plt.ylim(0.4, 1)
plt.errorbar(x=to_plot_on_2575['COF']['n_train'],
            y=to_plot_on_2575['COF']['r_square_avg'],
            yerr=to_plot_on_2575['COF']['r_square_std'],
            color=COF_color,
            fmt='o',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_2575['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_2575['intercept']['n_train'],
            y=to_plot_on_2575['intercept']['r_square_avg'],
            yerr=to_plot_on_2575['intercept']['r_square_std'],
            color=F0_color,
            fmt='s',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_2575['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)


legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/5050_on_2575.pdf', dpi=500)


# Refined R<sup>2</sup> plot 
### Everything models on 2575 test set

In [ ]:
to_plot_on_2575 = {'COF': {'n_train': list(),
                           'r_square_avg': list(),
                           'r_square_std': list(),
                           'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['2575'][target].items():
        if 'everything' in model:
            to_plot_on_2575[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_2575[target]['r_square_std'].append(data['r_square_std'])
            to_plot_on_2575[target]['n_train'].append(data['n_train'])
        elif 'original' in model: 
            to_plot_on_2575[target]['base'] = data['r_square']
                
plt.figure(figsize=(9, 6))

plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0, 8500)
plt.ylim(0.4, 1)
plt.errorbar(x=to_plot_on_2575['COF']['n_train'],
            y=to_plot_on_2575['COF']['r_square_avg'],
            yerr=to_plot_on_2575['COF']['r_square_std'],
            color=COF_color,
            fmt='o',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_2575['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_2575['intercept']['n_train'],
            y=to_plot_on_2575['intercept']['r_square_avg'],
            yerr=to_plot_on_2575['intercept']['r_square_std'],
            color=F0_color,
            fmt='s',
            markersize=markersize,
            elinewidth=1)
plt.axhline(y=to_plot_on_2575['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)

legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/everything_on_2575.pdf', dpi=500)


# Refined R<sup>2</sup> plot 
### Everything model on everything test set

In [ ]:
to_plot_on_everything = {'COF': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['everything'][target].items():
        if ('everything' in model):
            to_plot_on_everything[target]['n_train'].append(data['n_train'])
            to_plot_on_everything[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_everything[target]['r_square_std'].append(data['r_square_std'])
        elif 'original' in model: 
            to_plot_on_everything[target]['base'] = data['r_square']
plt.figure(figsize=(9, 6))

#plt.title('Effect of Training Set Size', fontsize=20)
plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0,8500)
plt.ylim(0.4, 1)
plt.errorbar(x=to_plot_on_everything['COF']['n_train'],
            y=to_plot_on_everything['COF']['r_square_avg'],
            yerr=to_plot_on_everything['COF']['r_square_std'],
            fmt='o',
            markersize=markersize,
            color=COF_color,
            elinewidth=1)
plt.axhline(y=to_plot_on_everything['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_everything['intercept']['n_train'],
            y=to_plot_on_everything['intercept']['r_square_avg'],
            yerr=to_plot_on_everything['intercept']['r_square_std'],
            fmt='s',
            markersize=markersize,
            color=F0_color,
            elinewidth=1)
plt.axhline(y=to_plot_on_everything['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)

legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/everything_on_everything.pdf', dpi=500)


# Refined R<sup>2</sup> plot 
### 5050 model on everything test set

In [ ]:
to_plot_on_everything = {'COF': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None},
                   'intercept': {'n_train': list(),
                                 'r_square_avg': list(),
                                 'r_square_std': list(),
                                 'base': None}}
for target in ['COF', 'intercept']:
    for model, data in results['everything'][target].items():
        if ('5050' in model):
            to_plot_on_everything[target]['n_train'].append(data['n_train'])
            to_plot_on_everything[target]['r_square_avg'].append(data['r_square_avg'])
            to_plot_on_everything[target]['r_square_std'].append(data['r_square_std'])
        elif 'original' in model: 
            to_plot_on_everything[target]['base'] = data['r_square']
plt.figure(figsize=(9, 6))

plt.xlabel('N', style='italic')
plt.ylabel(r'$\mathbf{R^2}$')
plt.xlim(0, 3000)
plt.ylim(0.4, 1)
plt.errorbar(x=to_plot_on_everything['COF']['n_train'],
            y=to_plot_on_everything['COF']['r_square_avg'],
            yerr=to_plot_on_everything['COF']['r_square_std'],
            fmt='o',
            markersize=markersize,
            color=COF_color,
            elinewidth=1)
plt.axhline(y=to_plot_on_everything['COF']['base'], linestyle='--', linewidth=1, color=COF_color)

plt.errorbar(x=to_plot_on_everything['intercept']['n_train'],
            y=to_plot_on_everything['intercept']['r_square_avg'],
            yerr=to_plot_on_everything['intercept']['r_square_std'],
            fmt='s',
            markersize=markersize,
            color=F0_color,
            elinewidth=1)
plt.axhline(y=to_plot_on_everything['intercept']['base'], linestyle='--', linewidth=1, color=F0_color)

legend_elements = [Line2D([0], [0], marker='o', color=COF_color, linestyle='', label='COF'),
                   Line2D([0], [0], marker='s', color=F0_color, linestyle='', label=r'$\mathit{F_0}$')]
plt.legend(handles=legend_elements, loc=4, prop={'size': 20})
plt.savefig(fname='./plots/r2vsN/5050_on_everything.pdf', dpi=500)


In [ ]:
to_plot_on_everything['COF']['n_train']

# Summary Table

In [ ]:
import pandas as pd

In [ ]:
pd_dict = {'model_name' : list(),
           'n_trains' : list(),
           '5050_COF' : list(),
           '5050_F0' : list(),
           '2575_COF' : list(),
           '2575_F0' : list(),
           'everything_COF' : list(),
           'everything_F0' : list()}

# Handle original models 
pd_dict['model_name'].append('original')
pd_dict['n_trains'].append(100)
for tset in ['5050', '2575', 'everything']:
    pd_dict[f'{tset}_COF'].append(round(results[tset]['COF']['original']['r_square'], 3))
    pd_dict[f'{tset}_F0'].append(round(results[tset]['intercept']['original']['r_square'], 3))


# Handle 5050 models
for point in mpoints:
    model_name = f'5050_{point}'
    pd_dict['model_name'].append(model_name)
    pd_dict['n_trains'].append(point)
    for tset in ['5050', '2575', 'everything']:
        COF = f"{round(results[tset]['COF'][model_name]['r_square_avg'], 3)} \u00B1 {round(results[tset]['COF'][model_name]['r_square_std'], 3)}"
        F0 = f"{round(results[tset]['intercept'][model_name]['r_square_avg'], 3)} \u00B1 {round(results[tset]['intercept'][model_name]['r_square_std'], 3)}"

        pd_dict[f'{tset}_COF'].append(COF)
        pd_dict[f'{tset}_F0'].append(F0)
        
        
# Handle 2575 models

for point in epoints:
    model_name = f'everything_{point}'
    pd_dict['model_name'].append(model_name)
    pd_dict['n_trains'].append(point)
    for tset in ['5050', '2575', 'everything']:
        COF = f"{round(results[tset]['COF'][model_name]['r_square_avg'], 3)} \u00B1 {round(results[tset]['COF'][model_name]['r_square_std'], 3)}"
        F0 = f"{round(results[tset]['intercept'][model_name]['r_square_avg'], 3)} \u00B1 {round(results[tset]['intercept'][model_name]['r_square_std'], 3)}"

        pd_dict[f'{tset}_COF'].append(COF)
        pd_dict[f'{tset}_F0'].append(F0)

In [ ]:
df = pd.DataFrame(pd_dict)

In [ ]:
#df.to_csv('plots/summary_r2.csv')